In [3]:
import gensim,logging
import os
import sys
from os import listdir,makedirs
from os.path import isfile, join,exists
import json
import spacy
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict,Counter
from nltk.corpus import stopwords
import string
import nltk
import numpy as np
import codecs
import random

/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [15]:
# Full cofea dataset
cofea_dir = join('data',"cofea_full")
cofea_files = [f for f in listdir(cofea_dir) if isfile(join(cofea_dir, f)) and ".json" in f]
cofea_full = []
for file in cofea_files: # adjust index slice here for the desired source
    if file  == 'evans_output.json':
        with open(cofea_dir+"/"+file, encoding = 'utf-8') as f:
            data = json.load(f)
            cofea_full.append(data)

In [5]:
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.tokenizer
stop = stopwords.words('english')

In [16]:
# new split
# time_periods = [1640,1740,1840,2010]
time_range = [[1640,1720],[1730,1810]] # want to sample 3520 docs to 1261. Actually, lets do tokens want 21972686 tokens
file_loc = join(cofea_dir,'range_data')
# update to generate time_range from the time periods to avoid manual errors
save_name = 'evans_even_sample_phrase_'

In [17]:
# make this stuff work with sources too
docs = defaultdict(list)
for file in cofea_full:
    for doc in file:
        if 'decade' in doc.keys():
            for start, end in time_range:
                #print(doc['decade'])
                #print(doc['id'])
                if start <= int(doc['decade']) <= end:
                    docs[str(start)+'_'+str(end)].append(doc['body'])

In [8]:
def sample_data(documents,sample_size,sample_tokens=False):
    random.seed(42)
    if sample_tokens:
        # TO-DO shuffle the documents, then select only enough documents to cover the desired token count
        sample = documents
    else:
        # sample docs
        sample = random.sample(documents,k=sample_size)
    return sample

In [9]:
#sample the data sets as needed
docs['1730_1810'] = sample_data(docs['1730_1810'],1261)

In [18]:
phrases = ['necessary and proper','cruel and unusual','bear arms','natural born','public use']

In [19]:
def clean_doc(doc,p_check=False):
    # break documents up into sentences and make sure each token is separated by just one space
    doc = doc.strip()
    doc = re.sub('(\\n|\\t|\\s)+'," ",doc)
    if p_check:
        for p in phrases:
            psub = p.replace(' ','')
            doc = doc.replace(p,psub)
    sents = sent_tokenize(doc)
    sents = [ ' '.join([y.text for y in tokenizer(x)]).lower() for x in sents] 
    return sents

In [20]:
for range_name in docs: 
    # clean up the docs for processing
    sents = []
    for d in docs[range_name]:
        sents.extend(clean_doc(d,True))
    fname = save_name +range_name+'.txt' #change name based on source
    # Save to text file so it can be efficiently used by word2vec
    with open(join(file_loc,fname), 'w',encoding = 'utf-8') as filehandle:
        for s in sents:
            filehandle.write('%s\n' % s)

In [21]:
def run_and_save(fname):
    
    #train model
    model = gensim.models.Word2Vec( alpha=0.025, window=4,vector_size=300, min_count=5, workers=12, sg=1, hs=0, negative=5)
    model.build_vocab(gensim.models.word2vec.LineSentence(fname+'.txt'))
    model.train(gensim.models.word2vec.LineSentence(fname+'.txt'), total_examples=model.corpus_count, epochs=5)
    model.wv.save_word2vec_format(fname+ '.tmp')
    # save .wv.npy and .vocab
    vec = []
    w = codecs.open(fname + '.vocab', 'w', encoding='utf-8')
    vocab_size, embed_dim = None, None
    with codecs.open(fname + '.tmp', 'r', encoding='utf-8', errors='ignore') as r:
        for line in r:
            items = line.strip().split()
            if not vocab_size:
                assert(len(items) == 2)
                vocab_size, embed_dim = int(items[0]), int(items[1])
            else:
                assert(len(items) == embed_dim + 1)
                vec.append([float(item) for item in items[1:]])
                w.write('%s\n'%items[0])
    w.close()
    vec = np.array(vec, dtype=np.float)
    assert(vec.shape[0] == vocab_size)
    assert(vec.shape[1] == embed_dim)
    np.save(fname + '.wv.npy', vec)
    print('saved %s.wv.npy'%fname)
    print('saved %s.vocab'%fname)
    os.remove(fname + '.tmp')
    

In [22]:
# https://rare-technologies.com/word2vec-tutorial/
#sentences = MySentences('cofea_full') 
#model = gensim.models.Word2Vec(sentences)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
for start,end in time_range: # TODO :Update to use different split based on authors 
    range_name = str(start)+'_'+str(end)
    print(range_name)
    fname = save_name+range_name #change name based on source
    fname = join(file_loc,fname) 
    run_and_save(fname)

2021-09-02 12:22:21,686 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.025)', 'datetime': '2021-09-02T12:22:21.685688', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-09-02 12:22:21,688 : INFO : collecting all words and their counts
2021-09-02 12:22:21,693 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-02 12:22:21,864 : INFO : PROGRESS: at sentence #10000, processed 252683 words, keeping 13822 word types


1640_1720


2021-09-02 12:22:21,958 : INFO : PROGRESS: at sentence #20000, processed 427197 words, keeping 19550 word types
2021-09-02 12:22:22,080 : INFO : PROGRESS: at sentence #30000, processed 673584 words, keeping 23991 word types
2021-09-02 12:22:22,149 : INFO : PROGRESS: at sentence #40000, processed 757365 words, keeping 25111 word types
2021-09-02 12:22:22,233 : INFO : PROGRESS: at sentence #50000, processed 908625 words, keeping 27263 word types
2021-09-02 12:22:22,369 : INFO : PROGRESS: at sentence #60000, processed 1222916 words, keeping 33955 word types
2021-09-02 12:22:22,496 : INFO : PROGRESS: at sentence #70000, processed 1518657 words, keeping 37269 word types
2021-09-02 12:22:22,618 : INFO : PROGRESS: at sentence #80000, processed 1817130 words, keeping 41204 word types
2021-09-02 12:22:22,744 : INFO : PROGRESS: at sentence #90000, processed 2093807 words, keeping 45206 word types
2021-09-02 12:22:22,908 : INFO : PROGRESS: at sentence #100000, processed 2526216 words, keeping 522

2021-09-02 12:22:30,273 : INFO : PROGRESS: at sentence #740000, processed 19649138 words, keeping 206722 word types
2021-09-02 12:22:30,420 : INFO : PROGRESS: at sentence #750000, processed 19997640 words, keeping 208799 word types
2021-09-02 12:22:30,561 : INFO : PROGRESS: at sentence #760000, processed 20347229 words, keeping 210637 word types
2021-09-02 12:22:30,683 : INFO : PROGRESS: at sentence #770000, processed 20635243 words, keeping 212303 word types
2021-09-02 12:22:30,806 : INFO : PROGRESS: at sentence #780000, processed 20932090 words, keeping 213909 word types
2021-09-02 12:22:30,930 : INFO : PROGRESS: at sentence #790000, processed 21234379 words, keeping 215891 word types
2021-09-02 12:22:31,032 : INFO : PROGRESS: at sentence #800000, processed 21451221 words, keeping 216900 word types
2021-09-02 12:22:31,115 : INFO : collected 217350 word types from a corpus of 21613202 raw words and 805687 sentences
2021-09-02 12:22:31,116 : INFO : Creating a fresh vocabulary
2021-09-0

2021-09-02 12:23:20,538 : INFO : EPOCH 1 - PROGRESS: at 73.97% examples, 217909 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:23:21,545 : INFO : EPOCH 1 - PROGRESS: at 75.37% examples, 218017 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:23:22,585 : INFO : EPOCH 1 - PROGRESS: at 76.88% examples, 218298 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:23:23,671 : INFO : EPOCH 1 - PROGRESS: at 78.28% examples, 218359 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:23:24,744 : INFO : EPOCH 1 - PROGRESS: at 79.91% examples, 218556 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:23:25,750 : INFO : EPOCH 1 - PROGRESS: at 82.11% examples, 218772 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:23:26,793 : INFO : EPOCH 1 - PROGRESS: at 84.23% examples, 219018 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:23:27,798 : INFO : EPOCH 1 - PROGRESS: at 85.95% examples, 219168 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:23:28,807 : INFO : EPOCH 1 - PROGRESS: at 87.62% examples, 219347 words/s,

...

2021-09-02 12:51:26,236 : INFO : EPOCH 5 - PROGRESS: at 26.39% examples, 199950 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:51:27,247 : INFO : EPOCH 5 - PROGRESS: at 26.77% examples, 199990 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:51:28,291 : INFO : EPOCH 5 - PROGRESS: at 27.07% examples, 200001 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:51:29,300 : INFO : EPOCH 5 - PROGRESS: at 27.33% examples, 200002 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:51:30,314 : INFO : EPOCH 5 - PROGRESS: at 27.55% examples, 199722 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:51:31,329 : INFO : EPOCH 5 - PROGRESS: at 27.88% examples, 199877 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:51:32,387 : INFO : EPOCH 5 - PROGRESS: at 28.07% examples, 199668 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:51:33,388 : INFO : EPOCH 5 - PROGRESS: at 28.36% examples, 199644 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:51:34,422 : INFO : EPOCH 5 - PROGRESS: at 28.66% examples, 199756 words/s,

2021-09-02 12:52:40,779 : INFO : EPOCH 5 - PROGRESS: at 48.92% examples, 201117 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:52:41,794 : INFO : EPOCH 5 - PROGRESS: at 49.24% examples, 201220 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:52:42,891 : INFO : EPOCH 5 - PROGRESS: at 49.50% examples, 201046 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:52:43,915 : INFO : EPOCH 5 - PROGRESS: at 49.84% examples, 201203 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:52:44,963 : INFO : EPOCH 5 - PROGRESS: at 50.18% examples, 201146 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:52:46,004 : INFO : EPOCH 5 - PROGRESS: at 50.50% examples, 201146 words/s, in_qsize 22, out_qsize 1
2021-09-02 12:52:47,010 : INFO : EPOCH 5 - PROGRESS: at 50.82% examples, 201254 words/s, in_qsize 23, out_qsize 1
2021-09-02 12:52:48,034 : INFO : EPOCH 5 - PROGRESS: at 51.13% examples, 201286 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:52:49,105 : INFO : EPOCH 5 - PROGRESS: at 51.44% examples, 201284 words/s,

2021-09-02 12:53:55,560 : INFO : EPOCH 5 - PROGRESS: at 73.35% examples, 202135 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:53:56,571 : INFO : EPOCH 5 - PROGRESS: at 73.63% examples, 202211 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:53:57,616 : INFO : EPOCH 5 - PROGRESS: at 73.94% examples, 202145 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:53:58,830 : INFO : EPOCH 5 - PROGRESS: at 74.32% examples, 202143 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:53:59,836 : INFO : EPOCH 5 - PROGRESS: at 74.75% examples, 202254 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:54:00,850 : INFO : EPOCH 5 - PROGRESS: at 75.02% examples, 202190 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:54:01,852 : INFO : EPOCH 5 - PROGRESS: at 75.40% examples, 202318 words/s, in_qsize 24, out_qsize 0
2021-09-02 12:54:02,870 : INFO : EPOCH 5 - PROGRESS: at 75.73% examples, 202264 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:54:03,954 : INFO : EPOCH 5 - PROGRESS: at 75.98% examples, 202097 words/s,

2021-09-02 12:55:10,718 : INFO : EPOCH 5 - PROGRESS: at 99.62% examples, 202799 words/s, in_qsize 23, out_qsize 0
2021-09-02 12:55:11,632 : INFO : worker thread finished; awaiting finish of 11 more threads
2021-09-02 12:55:11,643 : INFO : worker thread finished; awaiting finish of 10 more threads
2021-09-02 12:55:11,650 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-09-02 12:55:11,658 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-09-02 12:55:11,668 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-09-02 12:55:11,694 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-09-02 12:55:11,711 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-09-02 12:55:11,737 : INFO : EPOCH 5 - PROGRESS: at 99.95% examples, 202911 words/s, in_qsize 4, out_qsize 1
2021-09-02 12:55:11,738 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-09-02 12:55:11,801 : INFO : worker thr

saved data/cofea_full/range_data/evans_even_sample_phrase_1730_1810.wv.npy
saved data/cofea_full/range_data/evans_even_sample_phrase_1730_1810.vocab


In [43]:
# want decade or by source
group = [1640,1650]
file_loc = join(cofea_dir,'decades')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
for x in group: # TODO :Update to use different split based on authors 
    print(x)
    fname = 'data_'+str(x)
    fname = join(file_loc,fname) 
    run_and_save(fname)

2021-07-20 17:19:37,419 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.025)', 'datetime': '2021-07-20T17:19:37.419540', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-07-20 17:19:37,428 : INFO : collecting all words and their counts
2021-07-20 17:19:37,430 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-20 17:19:37,547 : INFO : collected 10014 word types from a corpus of 142510 raw words and 4377 sentences
2021-07-20 17:19:37,549 : INFO : Creating a fresh vocabulary
2021-07-20 17:19:37,589 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2189 unique words (21.859396844417816%% of original 10014, drops 7825)', 'datetime': '2021-07-20T17:19:37.588944', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64

1640


2021-07-20 17:19:37,654 : INFO : sample=0.001 downsamples 66 most-common words
2021-07-20 17:19:37,656 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 84049.85030880866 word corpus (64.4%% of prior 130454)', 'datetime': '2021-07-20T17:19:37.656692', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-07-20 17:19:37,724 : INFO : estimated required memory for 2189 words and 300 dimensions: 6348100 bytes
2021-07-20 17:19:37,725 : INFO : resetting layer weights
2021-07-20 17:19:37,739 : INFO : Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2021-07-20T17:19:37.739874', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'build_vocab'}
2021-07-20 17:19:37,741 : INFO : Word2Vec lifecycle event {'msg': 'training model with 12 workers on 21

2021-07-20 17:19:40,888 : INFO : collecting all words and their counts
2021-07-20 17:19:40,889 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-20 17:19:41,010 : INFO : PROGRESS: at sentence #10000, processed 193795 words, keeping 12692 word types
2021-07-20 17:19:41,020 : INFO : collected 13069 word types from a corpus of 207984 raw words and 11096 sentences
2021-07-20 17:19:41,021 : INFO : Creating a fresh vocabulary
2021-07-20 17:19:41,055 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2747 unique words (21.019205754074527%% of original 13069, drops 10322)', 'datetime': '2021-07-20T17:19:41.055655', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-07-20 17:19:41,056 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 192252 word corpus (92.43595661204708%% of original 207984, drops 15732)',

saved data/cofea_full/decades/data_1640.wv.npy
saved data/cofea_full/decades/data_1640.vocab
1650


2021-07-20 17:19:41,106 : INFO : sample=0.001 downsamples 60 most-common words
2021-07-20 17:19:41,114 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 126729.27259809534 word corpus (65.9%% of prior 192252)', 'datetime': '2021-07-20T17:19:41.114694', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-07-20 17:19:41,174 : INFO : estimated required memory for 2747 words and 300 dimensions: 7966300 bytes
2021-07-20 17:19:41,175 : INFO : resetting layer weights
2021-07-20 17:19:41,182 : INFO : Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2021-07-20T17:19:41.182869', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'build_vocab'}
2021-07-20 17:19:41,184 : INFO : Word2Vec lifecycle event {'msg': 'training model with 12 workers on 2

saved data/cofea_full/decades/data_1650.wv.npy
saved data/cofea_full/decades/data_1650.vocab
